<a href="https://colab.research.google.com/github/graylan0/mode-goat/blob/main/Llama2_Movie_Director_V0_Test_Alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install fastapi
!pip install nest-asyncio
!pip install json
!pip install uvicorn


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [4]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-GGML/resolve/main/llama-2-7b.ggmlv3.q8_0.bin

--2023-09-11 07:34:12--  https://huggingface.co/TheBloke/Llama-2-7B-GGML/resolve/main/llama-2-7b.ggmlv3.q8_0.bin
Resolving huggingface.co (huggingface.co)... 108.138.94.97, 108.138.94.52, 108.138.94.27, ...
Connecting to huggingface.co (huggingface.co)|108.138.94.97|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f8/8d/f88de0c18630cb860e75d1dc208c88baf212ec49bb6ff8b476801b0901ced546/5bb0702855f0c8abc645ea68c4d41e05207964ff54dd38c2787c1c1206cae121?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b.ggmlv3.q8_0.bin%3B+filename%3D%22llama-2-7b.ggmlv3.q8_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1694676852&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5NDY3Njg1Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9mOC84ZC9mODhkZTBjMTg2MzBjYjg2MGU3NWQxZGMyMDhjODhiYWYyMTJlYzQ5YmI2ZmY4YjQ3NjgwMWIwOTAxY2VkNTQ2LzViYjA

In [2]:
# Build Llama cpp
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=5822302 sha256=5bf25477ea5e670a5ddc8cda8920435365d45369e94b072fd1df10b3679ae36e
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python


In [ ]:
import asyncio
import json
import logging
from concurrent.futures import ThreadPoolExecutor
from llama_cpp import Llama  # Assuming you have this package installed

# Initialize logging to debug level to capture detailed logs
logging.basicConfig(level=logging.DEBUG)

llm = Llama(
  model_path="llama-2-7b.ggmlv3.q8_0.bin",
  n_gpu_layers=24,
  n_ctx=4000,
)

executor = ThreadPoolExecutor(max_workers=3)

# Function to trim tokens in a string to fit within a given limit
def trim_tokens(text, max_tokens):
    tokens = text.split()
    while len(tokens) > max_tokens:
        tokens.pop(0)
    return ' '.join(tokens)

# Function to extract key topics from a frame
def extract_key_topics(frame):
    return ' '.join(frame.split()[-3:])

async def continue_next_frame_generation(last_three_frames):
    key_topics = [extract_key_topics(frame) for frame in last_three_frames]
    combined_key_topics = ' '.join(key_topics)
    rules_prompt = f"As an AI specialized in Advanced Space Movies, you are tasked with generating a scene description based on these key topics: {combined_key_topics}"
    new_frame_generation = llm(rules_prompt, max_tokens=198)['choices'][0]['text']
    return new_frame_generation

async def generate_advanced_space_scene():
    rules_prompt = "As an AI specialized in Advanced Space Movies, generate an 18-word description of an advanced space scene."
    scene_output = llm(rules_prompt, max_tokens=150)['choices'][0]['text']
    return scene_output

async def start_movie(topic):
    initial_prompt = llm(f"Create a writing story prompt to start a Multiverse Movie Generator Game about {topic}.", max_tokens=312)['choices'][0]['text']
    frames = {}
    frames["frame_0"] = initial_prompt
    last_three_frames = [initial_prompt, "", ""]

    for i in range(1, 3):
        advanced_space_scene = await generate_advanced_space_scene()
        new_frame_generation = await continue_next_frame_generation([initial_prompt, advanced_space_scene, ""])
        frames[f"frame_{i}"] = new_frame_generation
        last_three_frames.pop(0)
        last_three_frames.append(new_frame_generation)

    sanitized_topic = ''.join(e for e in topic if e.isalnum())
    sanitized_topic = sanitized_topic[:50]

    with open(f"{sanitized_topic}_movie_frames.json", "w") as f:
        json.dump(frames, f, indent=4)

    return f"Advanced Space Movie about {topic} started and 500 frames generated. Saved to {sanitized_topic}_movie_frames.json"

if __name__ == "__main__":
    asyncio.run(start_movie("StarshipTroopersMovie"))


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
